In [1]:
import pandas as pd

# Carregando os filmes
filmes = pd.read_csv("dataset/small/movies.csv")
filmes.columns = ['filmeId', 'titulo','generos'] 
filmes = filmes.set_index("filmeId")
filmes.head()

,titulo,generos
filmeId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [2]:
# Carregando as notas
notas = pd.read_csv("dataset/small/ratings.csv")
notas.columns = ['usuarioId','filmeId','nota', 'momento'] 
notas.head()

,usuarioId,filmeId,nota,momento
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
notas.describe()

,usuarioId,filmeId,nota,momento
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


## Primeira tentativa de recomendação: heurística de total de votos

#### Não sei dado algum sobre o usuário, mas sei sobre os filmes

In [4]:
# Pego os filmes e ordeno pelo número de avaliações
total_de_votos = notas['filmeId'].value_counts()
total_de_votos.head()

356     329
318     317
296     307
593     279
2571    278
Name: filmeId, dtype: int64

In [5]:
# Localizo o filme com o maior número de avaliações e concateno a coluna 'total de votos' na minha lista de filmes
filmes.loc[356]
filmes['total_de_votos'] = total_de_votos
filmes.head()

,titulo,generos,total_de_votos
filmeId,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0
2,Jumanji (1995),Adventure|Children|Fantasy,110.0
3,Grumpier Old Men (1995),Comedy|Romance,52.0
4,Waiting to Exhale (1995),Comedy|Drama|Romance,7.0
5,Father of the Bride Part II (1995),Comedy,49.0


In [6]:
"""
 Ordenando os filmes pelo total de votos
 É a primeira heurísca, tentar recomendar pelo total de votos visto que não sei 
 Nada sobre meus usuários (inicialmente)
"""

filmes.sort_values('total_de_votos', ascending = False).head(10)

,titulo,generos,total_de_votos
filmeId,,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,251.0
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,238.0
110,Braveheart (1995),Action|Drama|War,237.0
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,224.0


In [7]:
# Tirando a nota média dos filmes
notas_medias = notas.groupby('filmeId').mean()['nota']
notas_medias.head()

filmeId
1    3.920930
2    3.431818
3    3.259615
4    2.357143
5    3.071429
Name: nota, dtype: float64

In [8]:
# Concatenando as notas médias nos filmes e exibindo os 10 primeiros filmes ordenados por  total de votos
filmes['notas_medias'] = notas_medias
filmes.sort_values('total_de_votos', ascending = False).head(10)

,titulo,generos,total_de_votos,notas_medias
filmeId,,,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0,4.164134
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0,4.197068
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0,4.161290
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0,4.192446
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,251.0,4.231076
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,238.0,3.750000
110,Braveheart (1995),Action|Drama|War,237.0,4.031646
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,224.0,3.970982


## Uma segunda heurística

#### Ordenar por total de votos não representa uma boa forma, visto que diz somente o número de avaliações, e não se um filme foi bem avaliado. Dessa forma é interessante ordenar pela nota média, nossa segunda heurística


In [9]:
"""
Aqui temos outro problema: apesar de os primeiros filmes serem todos nota 5, eles têm apenas 1 voto.
Logo, não é interessante considerar somente as notas médias. A classificação é mais complexa que isso
"""
filmes.sort_values('notas_medias', ascending = False).head(10)

,titulo,generos,total_de_votos,notas_medias
filmeId,,,,
88448,Paper Birds (Pájaros de papel) (2010),Comedy|Drama,1.0,5.0
100556,"Act of Killing, The (2012)",Documentary,1.0,5.0
143031,Jump In! (2007),Comedy|Drama|Romance,1.0,5.0
143511,Human (2015),Documentary,1.0,5.0
143559,L.A. Slasher (2015),Comedy|Crime|Fantasy,1.0,5.0
6201,Lady Jane (1986),Drama|Romance,1.0,5.0
102217,Bill Hicks: Revelations (1993),Comedy,1.0,5.0
102084,Justice League: Doom (2012),Action|Animation|Fantasy,1.0,5.0
6192,Open Hearts (Elsker dig for evigt) (2002),Romance,1.0,5.0


In [10]:
"""
Então fazemos uma query considerando filmes com no mínimo 50 votos. Ainda haverão filmes de nicho, mas temos que 
fazer um 'balanceamento' na heurísica considerando todas as caractestísticas.
"""
filmes_com_mais_de_50_votos = filmes.query("total_de_votos >= 50")
filmes_com_mais_de_50_votos.sort_values('notas_medias', ascending = False).head(10)

,titulo,generos,total_de_votos,notas_medias
filmeId,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
858,"Godfather, The (1972)",Crime|Drama,192.0,4.289062
2959,Fight Club (1999),Action|Crime|Drama|Thriller,218.0,4.272936
1276,Cool Hand Luke (1967),Drama,57.0,4.271930
750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,97.0,4.268041
904,Rear Window (1954),Mystery|Thriller,84.0,4.261905
1221,"Godfather: Part II, The (1974)",Crime|Drama,129.0,4.259690
48516,"Departed, The (2006)",Crime|Drama|Thriller,107.0,4.252336
1213,Goodfellas (1990),Crime|Drama,126.0,4.250000


In [11]:
"""
Vamos supor que alguns filmes foram assistidos, no intuito de supor que sabemos algo sobre o usuário
para podermos recomendar algo.
"""
filmes_assistidos = [1,21,19,10,11,7,2]
filmes.loc[filmes_assistidos]

,titulo,generos,total_de_votos,notas_medias
filmeId,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0,3.920930
21,Get Shorty (1995),Comedy|Crime|Thriller,89.0,3.494382
19,Ace Ventura: When Nature Calls (1995),Comedy,88.0,2.727273
10,GoldenEye (1995),Action|Adventure|Thriller,132.0,3.496212
11,"American President, The (1995)",Comedy|Drama|Romance,70.0,3.671429
7,Sabrina (1995),Comedy|Romance,54.0,3.185185
2,Jumanji (1995),Adventure|Children|Fantasy,110.0,3.431818


In [12]:
"""
E então fazer uma recomendação para o usuário. Nesse caso, iremos considerar o filme Jumanji, de índice 2:
"""

# Pegamos os filmes com os mesmos gêneros de Jumani: 'Adventure|Children|Fantasy'
aventura_infantial_e_fantasia = filmes_com_mais_de_50_votos.query("generos=='Adventure|Children|Fantasy'")

# E para evitarmos erros no pandas, 'dropamos' (removemos) os filmes que já foram assistidos
aventura_infantial_e_fantasia.drop(filmes_assistidos, errors='ignore').sort_values('notas_medias', ascending = False).head(10)

,titulo,generos,total_de_votos,notas_medias
filmeId,,,,
4896,Harry Potter and the Sorcerer's Stone (a.k.a. ...,Adventure|Children|Fantasy,107.0,3.761682
41566,"Chronicles of Narnia: The Lion, the Witch and ...",Adventure|Children|Fantasy,62.0,3.443548


In [13]:
"""
Colaborative filtering vs content based filtering (abordar sobre) & reccomender system
""" 

'\nColaborative filtering vs content based filtering (abordar sobre) & reccomender system\n'

## Procurando usuários "similares"


In [14]:
"""
Podemos calcular a distância euclidiana entre dois pontos (usuários) para tentar montar outro tipo de
sistema de recomendação
"""

import numpy as np


def distancia_de_vetores(a,b):
    return np.linalg.norm(a-b)

In [15]:
def notas_do_usuario(usuario):
    notas_do_usuario = notas.query("usuarioId==%d" % usuario)
    notas_do_usuario = notas_do_usuario[['filmeId', 'nota']].set_index('filmeId')
    return notas_do_usuario

In [16]:
"""
Pegamos dois usuários quaisquer e compararmos os filmes que eles assistiram. O .dropna() dropa todas as linhas 
com NaN, ou seja, filmes que somente um dos usuários assistiu
"""
usuario1 = notas_do_usuario(1)
usuario3 = notas_do_usuario(3)
diferencas = usuario1.join(usuario3, lsuffix='_esquerda', rsuffix='_direita').dropna()
distancia_de_vetores(diferencas['nota_esquerda'], diferencas['nota_direita'])

8.200609733428363

## Distância entre os usuários do dataset


In [17]:
# Determinando a distância entre 2 usuários
def distancia_de_usuarios(usuario_id_1, usuario_id_2):
    notas1 = notas_do_usuario(usuario_id_1)
    notas2 = notas_do_usuario(usuario_id_2)
    diferencas = notas1.join(notas2, lsuffix='_esquerda', rsuffix='_direita').dropna()
    return [usuario_id_1, usuario_id_2, distancia_de_vetores(diferencas['nota_esquerda'], diferencas['nota_direita'])]

In [18]:
"""
Apesar de ter o mesmo nome da função acima, aqui fazemos algumas alterações:
Na função acima, pessoas que não assistiram nenhum filme em comum têm a distância = 0, o que pode 
prejudar nossa análise. Iremos fazer as alterações para corrigir esse problema.
"""

def distancia_de_usuarios(usuario_id_1, usuario_id_2, minimo = 5):
    notas1 = notas_do_usuario(usuario_id_1)
    notas2 = notas_do_usuario(usuario_id_2)
    diferencas = notas1.join(notas2, lsuffix='_esquerda', rsuffix='_direita').dropna()
    
    if len(diferencas) < minimo: 
        return [usuario_id_1, usuario_id_2, 99999]
    
    return [usuario_id_1, usuario_id_2, distancia_de_vetores(diferencas['nota_esquerda'], diferencas['nota_direita'])]

In [19]:
distancia_de_usuarios(1,3)

[1, 3, 8.200609733428363]

In [20]:
"""
Sabendo a distância entre os usuários, podemos fazer um for entre eles e verificar quais as menores distâncias,
para podermos determinar os usuários mais próximos de um determinado usuário.
"""
quantidade_de_usuarios = len(notas['usuarioId'].unique())
print("Temos %d usuarios"%quantidade_de_usuarios)

Temos 610 usuarios


In [21]:
def distancia_entre_usuarios_dataset(usuario_base):
    todos_os_usuarios = notas['usuarioId'].unique()
    distancias = [distancia_de_usuarios(usuario_base, usuario_id) for usuario_id in todos_os_usuarios ]
    distancias = pd.DataFrame(distancias, columns = ['usuario_base', 'usuario_comparado', 'distancia'])
    return distancias

distancia_entre_usuarios_dataset(6)

,usuario_base,usuario_comparado,distancia
0,6,1,8.602325
1,6,2,99999.000000
2,6,3,99999.000000
3,6,4,8.426150
4,6,5,8.366600
...,...,...,...
605,6,606,9.539392
606,6,607,6.633250
607,6,608,17.066048
608,6,609,5.000000


In [22]:
# Para determinarmos a distância entre o usuário base (que definimos) e os demais usuários do dataset
def mais_proximos_de(usuario_base):
    distancias = distancia_entre_usuarios_dataset(usuario_base)
    distancias = distancias.sort_values("distancia")
    
    # E removemos o próprio usuário base:
    distancias = distancias.set_index("usuario_comparado").drop(usuario_base)
    return distancias

In [23]:
mais_proximos_de(1)

,usuario_base,distancia
usuario_comparado,,
77,1,0.000000
511,1,0.500000
366,1,0.707107
523,1,1.000000
49,1,1.000000
...,...,...
190,1,99999.000000
60,1,99999.000000
576,1,99999.000000


## Parâmetros de teste 


In [24]:
# Retorno None agora se os usuários não tem nada em comum
def distancia_de_usuarios(usuario_id_1, usuario_id_2, minimo = 5):
    notas1 = notas_do_usuario(usuario_id_1)
    notas2 = notas_do_usuario(usuario_id_2)
    diferencas = notas1.join(notas2, lsuffix='_esquerda', rsuffix='_direita').dropna()
    
    if len(diferencas) < minimo: 
        return None
    
    return [usuario_id_1, usuario_id_2, distancia_de_vetores(diferencas['nota_esquerda'], diferencas['nota_direita'])]

# Filtro e removo os usuários com none e posso limitar a n usuários
def distancia_entre_usuarios_dataset(usuario_base, numero_de_usuarios_a_analisar = None):
    todos_os_usuarios = notas['usuarioId'].unique()
    if numero_de_usuarios_a_analisar: 
        todos_os_usuarios = todos_os_usuarios[:numero_de_usuarios_a_analisar]
    distancias = [distancia_de_usuarios(usuario_base, usuario_id) for usuario_id in todos_os_usuarios ]
    distancias = list(filter(None, distancias))
    distancias = pd.DataFrame(distancias, columns = ['usuario_base', 'usuario_comparado', 'distancia'])
    return distancias


def mais_proximos_de(usuario_base, numero_de_usuarios_a_analisar = None):
    distancias = distancia_entre_usuarios_dataset(usuario_base, numero_de_usuarios_a_analisar = numero_de_usuarios_a_analisar)
    distancias = distancias.sort_values("distancia")
    distancias = distancias.set_index("usuario_comparado").drop(usuario_base)
    return distancias

In [25]:
mais_proximos_de(2, numero_de_usuarios_a_analisar = 50)

,usuario_base,distancia
usuario_comparado,,
16,2,2.000000
17,2,2.179449
29,2,2.179449
25,2,2.236068
24,2,2.291288
28,2,2.397916
22,2,3.000000
30,2,3.391165
21,2,4.153312


In [26]:
"""
Agora iremos pegar o usuário mais próximo do usuário base e a partir dele recomendar alguns filmes:
"""

def sugere_para(usuario_base, numero_de_usuarios_a_analisar = None):
    # Pego as notas do usuario base
    notas_usuario_base = notas_do_usuario(usuario_base)
    
    # Pego os filmes já vistos pelo usuario base, pois esses não precisam ser recomendados
    filmes_ja_vistos = notas_usuario_base.index
    
    # Determino as pessoas com gostos similares ao usuario base
    similares = mais_proximos_de(usuario_base, numero_de_usuarios_a_analisar = numero_de_usuarios_a_analisar)
    
    # E pego a pessoa mais similar ao usuário base
    similar = similares.iloc[0].name
    
    # Pego as notas que a pessoa mais próxima do usuário base deu aos filmes
    notas_do_similar = notas_do_usuario(similar)
    
    # E removo os filmes já vistos pelo usuário base
    notas_do_similar = notas_do_similar.drop(filmes_ja_vistos, errors = 'ignore')
    
    # Ordeno as recomendações de filmes
    recomendacoes = notas_do_similar.sort_values('nota', ascending = False)
    
    # E retorno as recomendações, juntando o nome dos filmes
    return recomendacoes.join(filmes)

sugere_para(1).head()

,nota,titulo,generos,total_de_votos,notas_medias
filmeId,,,,,
8636,5.0,Spider-Man 2 (2004),Action|Adventure|Sci-Fi|IMAX,79.0,3.803797
58559,5.0,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX,149.0,4.238255
33794,5.0,Batman Begins (2005),Action|Crime|IMAX,116.0,3.862069
4993,5.0,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy,198.0,4.106061
5349,5.0,Spider-Man (2002),Action|Adventure|Sci-Fi|Thriller,122.0,3.540984


## Sugerindo baseado em vários usuários

In [27]:
# Adicionamos o parâmetro n_mais_proximos
def mais_proximos_de(usuario_base, n_mais_proximos = 10,numero_de_usuarios_a_analisar = None):
    distancias = distancia_entre_usuarios_dataset(usuario_base, numero_de_usuarios_a_analisar = numero_de_usuarios_a_analisar)
    distancias = distancias.sort_values("distancia")
    distancias = distancias.set_index("usuario_comparado").drop(usuario_base)
    return distancias.head(n_mais_proximos)

In [28]:
mais_proximos_de(1, 3 ,numero_de_usuarios_a_analisar=300)

,usuario_base,distancia
usuario_comparado,,
77,1,0.0
258,1,1.0
49,1,1.0


In [29]:
"""
Iremos refatorar a função sugere_para para que ela aceite os n_mais_proximos (10 por padrão),
de forma que filmes de nicho não tenham uma relevância absurda e de forma a otimizar as recomendações
"""

def sugere_para(usuario_base, n_mais_proximos = 10, numero_de_usuarios_a_analisar = None):
    notas_usuario_base = notas_do_usuario(usuario_base)
    filmes_ja_vistos = notas_usuario_base.index
    similares = mais_proximos_de(usuario_base, n_mais_proximos = n_mais_proximos, numero_de_usuarios_a_analisar = numero_de_usuarios_a_analisar)

    # Pego os índices dos usuarios similares
    usuarios_similares = similares.index

    # Pego todas as notas dos similares com base no índice anterior
    notas_dos_similares = notas.set_index('usuarioId').loc[usuarios_similares]

    # Agrupo pelo id do filme (.groupby()),
    # Tiro a média das notas por filme (.mean()), 
    # E insiro num dataframe [['nota']]
    recomendacoes = notas_dos_similares.groupby("filmeId").mean()[['nota']]

    # Ordeno pelo valor da nota, do maior pro menor
    recomendacoes = recomendacoes.sort_values('nota', ascending=False)

    # E junto os filmes às suas notas, retornando o dataframe
    return recomendacoes.join(filmes)

In [30]:
sugere_para(1)

,nota,titulo,generos,total_de_votos,notas_medias
filmeId,,,,,
1704,5.0,Good Will Hunting (1997),Drama|Romance,141.0,4.078014
57504,5.0,"Girl Who Leapt Through Time, The (Toki o kaker...",Animation|Comedy|Drama|Romance|Sci-Fi,10.0,4.100000
38304,5.0,No Direction Home: Bob Dylan (2005),Documentary,4.0,4.625000
48394,5.0,"Pan's Labyrinth (Laberinto del fauno, El) (2006)",Drama|Fantasy|Thriller,81.0,3.814815
2300,5.0,"Producers, The (1968)",Comedy,33.0,3.969697
...,...,...,...,...,...
5507,1.0,xXx (2002),Action|Crime|Thriller,24.0,2.770833
4131,1.0,Making Mr. Right (1987),Comedy|Romance|Sci-Fi,3.0,1.833333
5891,1.0,I Spit on Your Grave (Day of the Woman) (1978),Horror|Thriller,4.0,2.625000


In [31]:
"""
Basicamente fizemos uma implementação da ideia do algorítimo 'k nearest neighbors':
"""

def knn(usuario_base, k_mais_proximos = 10,numero_de_usuarios_a_analisar = None):
    distancias = distancia_entre_usuarios_dataset(usuario_base, numero_de_usuarios_a_analisar = numero_de_usuarios_a_analisar)
    distancias = distancias.sort_values("distancia")
    distancias = distancias.set_index("usuario_comparado").drop(usuario_base)
    return distancias.head(k_mais_proximos)

def sugere_para(usuario_base, k_mais_proximos = 10, numero_de_usuarios_a_analisar = None):
    notas_usuario_base = notas_do_usuario(usuario_base)
    filmes_ja_vistos = notas_usuario_base.index
    
    similares = knn(usuario_base, k_mais_proximos = k_mais_proximos, numero_de_usuarios_a_analisar = numero_de_usuarios_a_analisar)
    usuarios_similares = similares.index
    notas_dos_similares = notas.set_index('usuarioId').loc[usuarios_similares]
    recomendacoes = notas_dos_similares.groupby("filmeId").mean()[['nota']]
    recomendacoes = recomendacoes.sort_values('nota', ascending=False)
    return recomendacoes.join(filmes)

## Testando um novo usuário

In [32]:
filmes.loc[[122904, 1246, 2529, 2329, 2324, 1, 7, 2, 1196, 260]]

,titulo,generos,total_de_votos,notas_medias
filmeId,,,,
122904,Deadpool (2016),Action|Adventure|Comedy|Sci-Fi,54.0,3.833333
1246,Dead Poets Society (1989),Drama,86.0,3.959302
2529,Planet of the Apes (1968),Action|Drama|Sci-Fi,56.0,3.803571
2329,American History X (1998),Crime|Drama,129.0,4.217054
2324,Life Is Beautiful (La Vita è bella) (1997),Comedy|Drama|Romance|War,88.0,4.147727
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0,3.920930
7,Sabrina (1995),Comedy|Romance,54.0,3.185185
2,Jumanji (1995),Adventure|Children|Fantasy,110.0,3.431818
1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi,211.0,4.215640


In [33]:
"""
Iremos criar um método de inserção de um novo usuário, com dados no formato [['filmeId', 'nota'], ...]
"""

def novo_usuario(dados):
    # Pego o número de usuários totais e somo 1
    novo_usuario = notas['usuarioId'].max() + 1
    
    # Crio um dataframe com os dados recebidos
    notas_do_usuario_novo = pd.DataFrame(dados, columns=['filmeId', 'nota'])
    
    # Insiro o id do novo usuário nos dados
    notas_do_usuario_novo['usuarioId'] = novo_usuario
    
    return pd.concat([notas, notas_do_usuario_novo])

In [34]:
# E adicionamos o novo usuário nas notas:



dados = [
    [122904,2],
    [1246,5],
    [2529,2],
    [2329,5],
    [2324,5],
    [1,2],
    [7,0.5],
    [2,2],
    [1196,1],
    [260,1],
]

notas = novo_usuario(dados)
notas.tail()

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


,filmeId,momento,nota,usuarioId
5,1,NaN,2.0,611
6,7,NaN,0.5,611
7,2,NaN,2.0,611
8,1196,NaN,1.0,611
9,260,NaN,1.0,611


In [35]:
sugere_para(611).head()

,nota,titulo,generos,total_de_votos,notas_medias
filmeId,,,,,
171495,5.0,Cosmos,(no genres listed),2.0,4.500000
81847,5.0,Tangled (2010),Animation|Children|Comedy|Fantasy|Musical|Roma...,24.0,3.916667
1873,5.0,"Misérables, Les (1998)",Crime|Drama|Romance|War,10.0,3.750000
8014,5.0,"Spring, Summer, Fall, Winter... and Spring (Bo...",Drama,10.0,4.250000
103141,5.0,Monsters University (2013),Adventure|Animation|Comedy,16.0,3.875000


In [36]:
"""
Porém voltamos ao problema de termos filmes com poucos votos. Pode-se ver acima: nenhum dos 
filmes tem mais de 25 votos. Então vamos limitar as notas para somente filmes com mais de 50 votos.
"""

notas = notas.set_index('filmeId').loc[filmes_com_mais_de_50_votos.index]
notas.head()

,momento,nota,usuarioId
filmeId,,,
1,9.649827e+08,4.0,1
1,8.474350e+08,4.0,5
1,1.106636e+09,4.5,7
1,1.510578e+09,2.5,15
1,1.305696e+09,4.5,17


In [38]:
# Porém perdemos a coluna 'filmeId', então precisamos resetar o índice para tê-la de volta:
notas = notas.reset_index()
notas.head()

,index,filmeId,momento,nota,usuarioId
0,0,1,9.649827e+08,4.0,1
1,1,1,8.474350e+08,4.0,5
2,2,1,1.106636e+09,4.5,7
3,3,1,1.510578e+09,2.5,15
4,4,1,1.305696e+09,4.5,17


In [40]:
"""
Observe que agora os filmes já terão um total de votos >= 50, otimizando mais as recomendações
"""

sugere_para(611).head()

,nota,titulo,generos,total_de_votos,notas_medias
filmeId,,,,,
112852,5.000000,Guardians of the Galaxy (2014),Action|Adventure|Sci-Fi,59.0,4.050847
74458,4.750000,Shutter Island (2010),Drama|Mystery|Thriller,67.0,4.022388
71535,4.666667,Zombieland (2009),Action|Comedy|Horror,53.0,3.877358
78499,4.500000,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX,55.0,4.109091
2804,4.500000,"Christmas Story, A (1983)",Children|Comedy,55.0,3.972727


## Uma outra abordagem: remover os filmes com poucos votos entre os usuários similares

#### Dessa forma as sugestões ficam mais objetivas e menos enviesadas

In [55]:
"""
Vamos reescrever a função sugere_para
"""

def sugere_para(usuario_base, k_mais_proximos = 10, numero_de_usuarios_a_analisar = None):
    notas_usuario_base = notas_do_usuario(usuario_base)
    filmes_ja_vistos = notas_usuario_base.index
    
    similares = knn(usuario_base, k_mais_proximos = k_mais_proximos, numero_de_usuarios_a_analisar = numero_de_usuarios_a_analisar)
    usuarios_similares = similares.index
    notas_dos_similares = notas.set_index('usuarioId').loc[usuarios_similares]
    recomendacoes = notas_dos_similares.groupby("filmeId").mean()[['nota']]
    
    # Verifico o número de pessoas que assistiram o mesmo filme
    aparicoes = notas_dos_similares.groupby('filmeId').count()[['nota']]
    
    # Determino um limiar mínimo para avaliar esse filme (menos que isso o filme é descartado)
    filtro_minimo = k_mais_proximos / 2
    
    # Junto a coluna de apariçoes com as recomendações
    recomendacoes = recomendacoes.join(aparicoes, lsuffix='_media_dos_usuarios', rsuffix='_aparicoes_nos_usuarios')
    
    # E faço uma query removendo os filmes com o limiar de aparições menor que o mínimo
    recomendacoes = recomendacoes.query("nota_aparicoes_nos_usuarios >= %.2f" % filtro_minimo)
    
    # Por fim removo os filmes já vistos
    recomendacoes = recomendacoes.drop(filmes_ja_vistos, errors='ignore')
    
    recomendacoes = recomendacoes.sort_values('nota_media_dos_usuarios', ascending=False)
    return recomendacoes.join(filmes)




In [56]:
def knn(usuario_base, k_mais_proximos = 10,numero_de_usuarios_a_analisar = None):
    distancias = distancia_entre_usuarios_dataset(usuario_base, numero_de_usuarios_a_analisar = numero_de_usuarios_a_analisar)
    distancias = distancias.sort_values("distancia")
    distancias = distancias.set_index("usuario_comparado").drop(usuario_base, errors='ignore')
    return distancias.head(k_mais_proximos)

In [57]:
sugere_para(611).head()

,nota_media_dos_usuarios,nota_aparicoes_nos_usuarios,titulo,generos,total_de_votos,notas_medias
filmeId,,,,,,
318,4.500000,8,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
919,4.400000,5,"Wizard of Oz, The (1939)",Adventure|Children|Fantasy|Musical,92.0,3.880435
79132,4.357143,7,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX,143.0,4.066434
356,4.350000,10,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0,4.164134
7361,4.312500,8,Eternal Sunshine of the Spotless Mind (2004),Drama|Romance|Sci-Fi,131.0,4.160305


In [58]:
sugere_para(611, k_mais_proximos=20).head(10)

,nota_media_dos_usuarios,nota_aparicoes_nos_usuarios,titulo,generos,total_de_votos,notas_medias
filmeId,,,,,,
593,4.266667,15,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0,4.161290
296,4.264706,17,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0,4.197068
4973,4.192308,13,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",Comedy|Romance,120.0,4.183333
318,4.176471,17,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
46578,4.150000,10,Little Miss Sunshine (2006),Adventure|Comedy|Drama,77.0,3.883117
590,4.136364,11,Dances with Wolves (1990),Adventure|Drama|Western,164.0,3.835366
7361,4.133333,15,Eternal Sunshine of the Spotless Mind (2004),Drama|Romance|Sci-Fi,131.0,4.160305
4878,4.125000,12,Donnie Darko (2001),Drama|Mystery|Sci-Fi|Thriller,109.0,3.981651
1213,4.100000,10,Goodfellas (1990),Crime|Drama,126.0,4.250000
